In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

mpg_df = sns.load_dataset('mpg')

In [6]:
pipe_horsepower = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

pipe_horsepower

,steps,"[('imputer', ...), ('scaler', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [9]:
pipe_horsepower.fit(mpg_df[['horsepower']])
pipe_horsepower


,steps,"[('imputer', ...), ('scaler', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [12]:
pipe_horsepower['scaler'].mean_

array([104.3040201])

In [14]:
pipe_horsepower['scaler'].n_samples_seen_

np.int64(398)

In [17]:
transformer_numeric = ColumnTransformer(
transformers = [
 ('with_nan', pipe_horsepower, ["horsepower"]),
     ('without_nan', StandardScaler(), ["mpg", "displacement","weight", "acceleration"]),
], # el resto por default se va a drop
   # remainder="passthrough"
)
transformer_numeric

,transformers,"[('with_nan', ...), ('without_nan', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [18]:
transformer_numeric.fit_transform(mpg_df)

array([[ 0.67311762, -0.7064387 ,  1.0906037 ,  0.63086987, -1.29549834],
       [ 1.58995818, -1.09075062,  1.5035143 ,  0.85433297, -1.47703779],
       [ 1.19702651, -0.7064387 ,  1.19623199,  0.55047045, -1.65857724],
       ...,
       [-0.53187283,  1.08701694, -0.56103873, -0.79858454, -1.4407299 ],
       [-0.66285006,  0.57460104, -0.70507731, -0.40841088,  1.10082237],
       [-0.58426372,  0.95891297, -0.71467988, -0.29608816,  1.39128549]],
      shape=(398, 5))

In [21]:
# vamos a crear otro mas grande
preprocessor =ColumnTransformer(
transformers = [
     ('numeric_with_nan', pipe_horsepower, ["horsepower"]),
     ('numeric_without_nan', StandardScaler(), ["mpg", "displacement","weight", "acceleration"]),
     ('categorical_ordinal', OrdinalEncoder(categories="auto"), ["cylinders", "model_year"]),
     ('categorical_nominal', OneHotEncoder(categories="auto", handle_unknown='error'), ["origin", "name"]),
],
remainder='drop'
)
preprocessor

,transformers,"[('numeric_with_nan', ...), ('numeric_without_nan', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [23]:
preprocessor.fit_transform(mpg_df).toarray()

array([[ 0.67311762, -0.7064387 ,  1.0906037 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.58995818, -1.09075062,  1.5035143 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.19702651, -0.7064387 ,  1.19623199, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.53187283,  1.08701694, -0.56103873, ...,  0.        ,
         0.        ,  0.        ],
       [-0.66285006,  0.57460104, -0.70507731, ...,  0.        ,
         0.        ,  0.        ],
       [-0.58426372,  0.95891297, -0.71467988, ...,  0.        ,
         0.        ,  0.        ]], shape=(398, 315))

In [25]:
pipe = Pipeline(steps=[
    ("preprocessor",preprocessor)
])

pipe.fit_transform(mpg_df).toarray()

array([[ 0.67311762, -0.7064387 ,  1.0906037 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.58995818, -1.09075062,  1.5035143 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.19702651, -0.7064387 ,  1.19623199, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.53187283,  1.08701694, -0.56103873, ...,  0.        ,
         0.        ,  0.        ],
       [-0.66285006,  0.57460104, -0.70507731, ...,  0.        ,
         0.        ,  0.        ],
       [-0.58426372,  0.95891297, -0.71467988, ...,  0.        ,
         0.        ,  0.        ]], shape=(398, 315))

In [26]:
# DataFrame amb el nombs de les comlumnes
resultat = pd.DataFrame(pipe.fit_transform(mpg_df).toarray())
resultat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,...,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314
0,0.673118,-0.706439,1.090604,0.630870,-1.295498,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.589958,-1.090751,1.503514,0.854333,-1.477038,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.197027,-0.706439,1.196232,0.550470,-1.658577,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.197027,-0.962647,1.061796,0.546923,-1.295498,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.935072,-0.834543,1.042591,0.565841,-1.840117,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,-0.479482,0.446497,-0.513026,-0.213324,0.011586,1.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
394,-1.370127,2

In [31]:
# lo siguietne es ponerle nombre a las columnas segun lo haya visto 
pipe['preprocessor']["categorical_nominal"].get_feature_names_out() # asi accedo al categorical_nominal y desde ahi puedo obtener los nomber del categorical nominal que crea una columna por valro



array(['origin_europe', 'origin_japan', 'origin_usa',
       'name_amc ambassador brougham', 'name_amc ambassador dpl',
       'name_amc ambassador sst', 'name_amc concord',
       'name_amc concord d/l', 'name_amc concord dl',
       'name_amc concord dl 6', 'name_amc gremlin', 'name_amc hornet',
       'name_amc hornet sportabout (sw)', 'name_amc matador',
       'name_amc matador (sw)', 'name_amc pacer', 'name_amc pacer d/l',
       'name_amc rebel sst', 'name_amc spirit dl', 'name_audi 100 ls',
       'name_audi 100ls', 'name_audi 4000', 'name_audi 5000',
       'name_audi 5000s (diesel)', 'name_audi fox', 'name_bmw 2002',
       'name_bmw 320i', 'name_buick century', 'name_buick century 350',
       'name_buick century limited', 'name_buick century luxus (sw)',
       'name_buick century special', 'name_buick electra 225 custom',
       'name_buick estate wagon (sw)', 'name_buick lesabre custom',
       'name_buick opel isuzu deluxe',
       'name_buick regal sport coupe (turbo)',

In [32]:
# ponemos la lista en orden
columnes =['horsepower',"mpg", "displacement","weight", "acceleration"]
columnes = np.append(columnes, ["cylinders", "model_year"])
columnes = np.append(columnes, pipe['preprocessor']["categorical_nominal"].get_feature_names_out())
columnes

array(['horsepower', 'mpg', 'displacement', 'weight', 'acceleration',
       'cylinders', 'model_year', 'origin_europe', 'origin_japan',
       'origin_usa', 'name_amc ambassador brougham',
       'name_amc ambassador dpl', 'name_amc ambassador sst',
       'name_amc concord', 'name_amc concord d/l', 'name_amc concord dl',
       'name_amc concord dl 6', 'name_amc gremlin', 'name_amc hornet',
       'name_amc hornet sportabout (sw)', 'name_amc matador',
       'name_amc matador (sw)', 'name_amc pacer', 'name_amc pacer d/l',
       'name_amc rebel sst', 'name_amc spirit dl', 'name_audi 100 ls',
       'name_audi 100ls', 'name_audi 4000', 'name_audi 5000',
       'name_audi 5000s (diesel)', 'name_audi fox', 'name_bmw 2002',
       'name_bmw 320i', 'name_buick century', 'name_buick century 350',
       'name_buick century limited', 'name_buick century luxus (sw)',
       'name_buick century special', 'name_buick electra 225 custom',
       'name_buick estate wagon (sw)', 'name_buick lesab

In [33]:
resultat.columns = columnes
resultat

,horsepower,mpg,displacement,weight,acceleration,cylinders,model_year,origin_europe,origin_japan,origin_usa,name_amc ambassador brougham,name_amc ambassador dpl,name_amc ambassador sst,name_amc concord,name_amc concord d/l,name_amc concord dl,name_amc concord dl 6,name_amc gremlin,name_amc hornet,name_amc hornet sportabout (sw),name_amc matador,name_amc matador (sw),name_amc pacer,name_amc pacer d/l,name_amc rebel sst,name_amc spirit dl,name_audi 100 ls,name_audi 100ls,name_audi 4000,name_audi 5000,name_audi 5000s (diesel),name_audi fox,name_bmw 2002,name_bmw 320i,name_buick century,name_buick century 350,name_buick century limited,name_buick century luxus (sw),name_buick century special,name_buick electra 225 custom,name_buick estate wagon (sw),name_buick lesabre custom,name_buick opel isuzu deluxe,name_buick regal sport coupe (turbo),name_buick skyhawk,name_buick skylark,name_buick skylark 320,name_buick skylark limited,name_cadillac eldorado,name_cadillac seville,name_capri ii,name_chevroelt chevelle malibu,name_chevrolet bel air,name_chevrolet camaro,name_chevrolet caprice classic,name_chevrolet cavalier,name_chevrolet cavalier 2-door,name_chevrolet cavalier wagon,name_chevrolet chevelle concours (sw),name_chevrolet chevelle malibu,...,name_pontiac grand prix lj,name_pontiac j2000 se hatchback,name_pontiac lemans v6,name_pontiac phoenix,name_pontiac phoenix lj,name_pontiac safari (sw),name_pontiac sunbird coupe,name_pontiac ventura sj,name_renault 12 (sw),name_renault 12tl,name_renault 18i,name_renault 5 gtl,name_renault lecar deluxe,name_saab 99e,name_saab 99gle,name_saab 99le,name_subaru,name_subaru dl,name_toyota carina,name_toyota celica gt,name_toyota celica gt liftback,name_toyota corolla,name_toyota corolla 1200,name_toyota corolla 1600 (sw),name_toyota corolla liftback,name_toyota corolla tercel,name_toyota corona,name_toyota corona hardtop,name_toyota corona liftback,name_toyota corona mark ii,name_toyota cressida,name_toyota mark ii,name_toyota starlet,name_toyota tercel,name_toyouta corona mark ii (sw),name_triumph tr7 coupe,name_vokswagen rabbit,name_volkswagen 1131 deluxe sedan,name_volkswagen 411 (sw),name_volkswagen dasher,name_volkswagen jetta,name_volkswagen model 111,name_volkswagen rabbit,name_volkswagen rabbit custom,name_volkswagen rabbit custom diesel,name_volkswagen rabbit l,name_volkswagen scirocco,name_volkswagen super beetle,name_volkswagen type 3,name_volvo 144ea,name_volvo 145e (sw),name_volvo 244dl,name_volvo 245,name_volvo 264gl,name_volvo diesel,name_vw dasher (diesel),name_vw pickup,name_vw rabbit,name_vw rabbit c (diesel),name_vw rabbit custom
0,0.673118,-0.706439,1.090604,0.630870,-1.295498,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.589958,-1.090751,1.503514,0.854333,-1.477038,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.197027,-0.706439,1.196232,0.550470,-1.658577,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.